Guilherme Henrique Silva Barbara - 202120499

# **1. Modelagem (Heurística Gulosa)**

    
**Raiz:** Encontrar a mochila fracionária para calcular o melhor limitante superior. Em seguida, calcular o limitante inferior com base em uma solução inicial que seja válida; 

**Expansão:** A expansão de nós será priorizando o melhor limitante superior. E, logo após isso, dividindo o problema em subproblemas; 

**Poda:** Não expandir nós cujo limitante superior seja menor ou igual ao melhor limitante inferior; 

**Conclusão:** E por fim, será retornado a melhor solução inteira encontrada.

### Saídas esperadas:

* Número de nós explorados; 

* GAP de otimalidade: (UB − LB)/(UB × 100). O qual, UB representa o limitante superior e LB o inferior;

* Tempo computacional (***time()***).


**Obs.** 
* Ordenar itens pelo valor por peso (vi/wi​) e adicionar à solução até que a capacidade seja atingida; 

* Selecionar itens na mesma ordem gulosa, mas sem fracionamento.


In [ ]:
import time

class MochilaBranchAndBound:
    def __init__(self, capacidade, pesos, valores):
        self.capacidade = capacidade
        self.pesos = pesos
        self.valores = valores
        self.qtdItens = len(pesos)
    
    def mochilaFracionaria(self, itensDisponiveis, capacidadeRestante):
        #Valor maixmo: mochila fracionaria
        itensOrdenados = sorted(itensDisponiveis, key=lambda x: self.valores[x] / self.pesos[x], reverse=True)
        valorTotal = 0
        for i in itensOrdenados:
            if self.pesos[i] <= capacidadeRestante:
                valorTotal += self.valores[i]
                capacidadeRestante -= self.pesos[i]
            else:
                valorTotal += self.valores[i] * (capacidadeRestante / self.pesos[i])
                break
        return valorTotal

    def branchAndBound(self):
        #Mochila binaria: Branch and Bound
        tempoInicio = time.time()
        nosExplorados = 0
        melhorSolucao = 0
        itensMelhorSolucao = []
        
        #Nó raiz
        pilha = [{'nivel': 0, 'valor': 0, 'peso': 0, 'limitante': self.mochilaFracionaria(range(self.qtdItens), self.capacidade), 'itens': []}]
        
        while pilha:
            noAtual = pilha.pop()
            nosExplorados += 1
            
            #Poda
            if noAtual['limitante'] <= melhorSolucao or noAtual['peso'] > self.capacidade:
                continue
            
            #Expansão
            nivel = noAtual['nivel']
            if nivel < self.qtdItens:
                proximosItens = noAtual['itens'] + [nivel]
                if noAtual['peso'] + self.pesos[nivel] <= self.capacidade:
                    novoValor = noAtual['valor'] + self.valores[nivel]
                    novoPeso = noAtual['peso'] + self.pesos[nivel]
                    if novoValor > melhorSolucao:
                        melhorSolucao = novoValor
                        itensMelhorSolucao = proximosItens
                    pilha.append({
                        'nivel': nivel + 1,
                        'valor': novoValor,
                        'peso': novoPeso,
                        'limitante': novoValor + self.mochilaFracionaria(range(nivel + 1, self.qtdItens), self.capacidade - novoPeso),
                        'itens': proximosItens
                    })
                
                pilha.append({
                    'nivel': nivel + 1,
                    'valor': noAtual['valor'],
                    'peso': noAtual['peso'],
                    'limitante': noAtual['valor'] + self.mochilaFracionaria(range(nivel + 1, self.qtdItens), self.capacidade - noAtual['peso']),
                    'itens': noAtual['itens']
                })
        
        tempoFim = time.time()
        
        #Otimalidade
        gap = (pilha[0]['limitante'] - melhorSolucao) / pilha[0]['limitante'] * 100 if pilha else 0
        
        return {
            'solucao': melhorSolucao,
            'itens': itensMelhorSolucao,
            'nosExplorados': nosExplorados,
            'gap': gap,
            'tempo': tempoFim - tempoInicio
        }

def lerInstancia(caminhoArquivo):
    with open(caminhoArquivo, 'r') as arquivo:
        linhas = arquivo.readlines()
        nomeInstancia = linhas[0].strip()
        capacidade = int(linhas[2].strip())
        pesos, valores = [], []
        for linha in linhas[3:]:
            peso, valor = map(int, linha.split())
            pesos.append(peso)
            valores.append(valor)
        return nomeInstancia, capacidade, pesos, valores

if __name__ == "__main__":
    nomeInstancia, capacidade, pesos, valores = lerInstancia("Weakly001")
    resolvedor = MochilaBranchAndBound(capacidade, pesos, valores)
    resultado = resolvedor.branchAndBound()
    print(f"Instância: {nomeInstancia}")
    print(f"Valor máximo: {resultado['solucao']}")
    print(f"Itens escolhidos: {resultado['itens']}")
    print(f"Nós explorados: {resultado['nosExplorados']}")
    print(f"GAP de otimalidade: {resultado['gap']:.2f}%")
    print(f"Tempo computacional: {resultado['tempo']:.2f}s")


Instância: Weakly001
Valor máximo: 4886
Itens escolhidos: [0, 3, 9, 12, 13, 30, 78, 102, 110, 139, 181, 218, 293, 295, 316, 328, 331, 369, 395, 470, 472, 496, 504, 511, 555, 557, 573, 590, 618, 631, 674, 686, 689, 699, 744, 751, 788, 792, 837, 841, 886, 901, 922, 930, 936, 954, 967, 978, 988, 989, 993]
Nós explorados: 6932431
GAP de otimalidade: 0.00%
Tempo computacional: 542.02s
